In [13]:
import numpy as np
import pandas as pd
import datetime
import math

In [14]:
daily=pd.read_csv('dam_daily.csv')
daily.columns

Index(['year', 'month', 'day', 'volume', 'value', 'price_weight_av',
       'price_max', 'price_min', 'price_mean', 'power_max', 'power_min',
       'power_mean', 'price_morning_max', 'price_evening_max',
       'price_standard', 'off_price_mean', 'standard_price_mean',
       'morning_price_mean', 'evening_price_mean', 'off_hour_count',
       'standard_hour_count', 'morning_hour_count', 'evening_hour_count',
       'off_volume', 'standard_volume', 'morning_volume', 'evening_volume',
       'off_value', 'standard_value', 'morning_value', 'evening_value'],
      dtype='object')

In [15]:
monthly=daily.groupby(['year','month']).mean().reset_index(['month','year'])
monthly=monthly.drop(['morning_volume','off_volume','standard_volume','evening_volume','day','price_weight_av'],1)
monthly.rename(columns={'value':'value_mean','volume':'volume_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)
monthly[['price_max','standard_price_max','off_price_max','morning_price_max','evening_price_max']]=daily.groupby(['year','month']).max().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_min','standard_price_min','off_price_min','morning_price_min','evening_price_min']]=daily.groupby(['year','month']).min().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]

monthly[['volume','morning_volume','evening_volume','standard_volume','off_volume',]]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['volume','morning_volume','evening_volume','standard_volume','off_volume']]
monthly[['value','morning_value','evening_value','standard_value','off_value',]]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['volume','morning_value','evening_value','standard_value','off_value']]

monthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]

monthly['price_weight_mean']=monthly['value'] / monthly['volume']
monthly['off_price_weight_mean']=monthly['off_value'] / monthly['off_volume']
monthly['off_price_weight_mean']=monthly['off_value'] / monthly['off_volume']
monthly['off_price_weight_mean']=monthly['off_value'] / monthly['off_volume']
monthly['off_price_weight_mean']=monthly['off_value'] / monthly['off_volume']

monthly['off_power']=monthly['off_volume'] / monthly['off_hour_count']
monthly['morning_power']=monthly['morning_volume'] / monthly['morning_hour_count']
monthly['standard_power']=monthly['standard_volume'] / monthly['standard_hour_count']
monthly['evening_power']=monthly['evening_volume'] / monthly['evening_hour_count']

monthly

,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,off_hour_count,standard_hour_count,morning_hour_count,evening_hour_count,price_weight_mean,off_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2016,1,3877.909677,216538.697903,104.036250,45.982500,81.784624,362.651613,17.606452,161.579570,...,373,266.0,63.0,42.0,1.0,50.385088,272.092493,33.409524,56.583459,37.350000
1,2016,2,3915.110345,263339.799483,111.843750,75.023333,91.385273,367.837931,23.231034,163.129598,...,332,259.0,63.0,42.0,1.0,56.542839,275.132530,12.280952,75.799614,42.580952
2,2016,3,3004.254839,192831.319548,101.464167,60.115833,88.099516,285.867742,18.232258,125.177285,...,348,281.0,69.0,46.0,1.0,55.124423,214.125862,15.502899,56.748043,34.786957
3,2016,4,1934.450000,119384.364433,114.641667,40.925417,85.320472,183.970000,7.236667,80.602083,...,349,266.0,63.0,42.0,1.0,56.919792,149.820630,7.793651,19.055263,4.438095
4,2016,5,1714.970968,95755.502839,104.820833,28.659583,78.638495,165.619355,4.661290,71.457124,...,364,270.0,66.0,44.0,1.0,49.682582,130.040385,10.831818,18.262963,4.170455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,2020,12,2765.732258,215368.490710,127.069167,45.683750,87.973105,188.409677,53.967742,115.238844,...,348,281.0,69.0,46.0,1.0,49.038696,134.756609,103.126087,102.248399,65.106522
60,2021,1,2506.932258,151566.968968,88.205417,44.173333,63.961895,185.074194,41.967742,104.455511,...,373,266.0,63.0,42.0,1.0,49.170530,113.052547,84.269841,104.337218,59.133333
61,2021,2,2225.446429,183252.046929,136.436250,43.510417,84.913795,141.325000,48.175000,92.726935,...,324,248.0,60.0,40.0,1.0,56.300449,92.895988,97.925000,96.015726,63.170000
62,2021,3,2799.748387,160399.166548,78.673750,27.023333,61.029073,189.854839,58.119355,116.656183,...,348,281.0,69.0,46.0,1.0,40.480969,122.156897,87.672464,122.960142,80.008696


In [16]:
monthly['evening_days']=daily.loc[daily['evening_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['day']
monthly['morning_days']=daily.loc[daily['morning_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['day']
monthly['days']=daily.groupby(['year','month']).count().reset_index(['month','year'])['day']
monthly

,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,evening_hour_count,price_weight_mean,off_price_weight_mean,off_power,morning_power,standard_power,evening_power,evening_days,morning_days,days
0,2016,1,3877.909677,216538.697903,104.036250,45.982500,81.784624,362.651613,17.606452,161.579570,...,42.0,1.0,50.385088,272.092493,33.409524,56.583459,37.350000,21,21,31
1,2016,2,3915.110345,263339.799483,111.843750,75.023333,91.385273,367.837931,23.231034,163.129598,...,42.0,1.0,56.542839,275.132530,12.280952,75.799614,42.580952,21,21,29
2,2016,3,3004.254839,192831.319548,101.464167,60.115833,88.099516,285.867742,18.232258,125.177285,...,46.0,1.0,55.124423,214.125862,15.502899,56.748043,34.786957,23,23,31
3,2016,4,1934.450000,119384.364433,114.641667,40.925417,85.320472,183.970000,7.236667,80.602083,...,42.0,1.0,56.919792,149.820630,7.793651,19.055263,4.438095,21,21,30
4,2016,5,1714.970968,95755.502839,104.820833,28.659583,78.638495,165.619355,4.661290,71.457124,...,44.0,1.0,49.682582,130.040385,10.831818,18.262963,4.170455,22,22,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,2020,12,2765.732258,215368.490710,127.069167,45.683750,87.973105,188.409677,53.967742,115.238844,...,46.0,1.0,49.038696,134.756609,103.126087,102.248399,65.106522,23,23,31
60,2021,1,2506.932258,151566.968968,88.205417,44.173333,63.961895,185.074194,41.967742,104.455511,...,42.0,1.0,49.170530,113.052547,84.269841,104.337218,59.133333,21,21,31
61,2021,2,2225.446429,183252.046929,136.436250,43.510417,84.913795,141.325000,48.175000,92.726935,...,40.0,1.0,56.300449,92.895988,97.925000,96.015726,63.170000,20,20,28
62,2021,3,2799.748387,160399.166548,78.673750,27.023333,61.029073,189.854839,58.119355,116.656183,...,46.0,1.0,40.480969,122.156897,87.672464,122.960142,80.008696,23,23,31


In [17]:
monthly=monthly.set_index(['year','month'])
monthly.to_csv('dam_monthly.csv')